In [3]:
class Note:
    
    def __init__(self, channel, pitch, velocity, timestamp, duration, prev_note_buffer):
        self.channel = channel
        self.pitch = pitch
        self.velocity = velocity
        
        # in MIDI 'ticks'
        self.timestamp = timestamp
        self.duration = duration
        self.prev_note_buffer = prev_note_buffer
        
        # rounding values for prediction
        self.velocity_round = 40
        self.duration_round = 20000
        
    def rounded(self):
        return Note(self.channel, self.pitch, self.velocity - (self.velocity % self.velocity_round), \
                    self.timestamp, self.duration - (self.duration % self.duration_round), self.prev_note_buffer)
        
    def __eq__(self, other):
        return self.channel == other.channel and self.pitch == other.pitch and self.velocity == other.velocity and \
            self.duration == other.duration and self.prev_note_buffer == other.prev_note_buffer
        
    def __repr__(self):
        return f"[Note {self.channel} {self.pitch} {self.velocity} {self.duration}]"
        
    def __hash__(self):
        return hash(str(self))

In [28]:
from collections import defaultdict
from random import randint, choices
from midiutil.MidiFile import MIDIFile

class ConwayGenerator:
    
    def __init__(self, init_state):
        
        self.init_state = init_state
        self.delta = [(-1, -1), (-1, 0), (-1, 1), (0, 1), (1, 1), (1, 0), (1, -1), (0, -1)]

        self.D = 0
        self.L = 1
        self.L_L = 2
        self.L_D = 3
        self.D_L = 4
        self.D_D = 5
        
    def get_copy_of_state(self, state):
        m, n = len(self.init_state), len(init_state[0])
        copy = [[0 for _ in range(n)] for _ in range(m)]
        
        for i in range(m):
            for j in range(n):
                copy[i][j] = state[i][j]
        
        return copy
        
    def generate_states(self, num_states):

        state_lst = []
        state = self.get_copy_of_state(self.init_state)
        
        for _ in range(num_states):
            state_lst.append(self.get_copy_of_state(state))
            state = self.get_next_state(state)
      
        return state_lst
    
    def get_next_state(self, board):
        
        next_state = self.get_copy_of_state(self.init_state)
        m = len(next_state)
        n = len(next_state[0])
        
        for i in range(m):
            for j in range(n):
                next_state[i][j] = self.update_cell(board, i, j, m, n)
               
        for i in range(m):
            for j in range(n):
                if next_state[i][j] == self.L_L or next_state[i][j] == self.D_L:
                    next_state[i][j] = self.L
                else:
                    next_state[i][j] = self.D
        
        return next_state
    
        
    def update_cell(self, board, r, c, m, n):
        
        alive = 0
        dead = 0
        
        for dr, dc in self.delta:
            nr = r + dr
            nc = c + dc
            
            if nr < 0 or nr >= m or nc < 0 or nc >= n:
                continue
                
            if self.is_alive(board, nr, nc):
                alive += 1
            else:
                dead += 1
                
        if self.is_alive(board, r, c):
            if alive == 2 or alive == 3:
                return self.L_L
            
            return self.L_D
        else:
            if alive == 3:
                return self.D_L
            
            return self.D_D
        
    def is_alive(self, board, r, c):
        return board[r][c] == self.L or board[r][c] == self.L_L or board[r][c] == self.L_D
    
    def get_transformed_state(self, state):
    
        new_state = self.get_copy_of_state(state)

        m = len(new_state)
        n = len(new_state[0])

        for i in range(m):
            for j in range(n):

                if new_state[i][j] == 0:
                    new_state[i][j] = 'O'
                else:
                    new_state[i][j] = 'X'

        return new_state

    def format_state(self, state):
        return '\n'.join([''.join([str(cell) for cell in row]) for row in state])


In [30]:
from mido import Message, MidiFile, MidiTrack

init_state = [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0], 
              [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0],
              [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0],
              [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0],
              [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0],
              [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
              [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
              [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
              [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
              [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
              [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]

'''
Japanese Pentatonic Scale

E, F, A, B, C, E

MIDI

E: 52, 64, 76
F: 53, 65
A: 57, 69
B: 59, 71
C: 60, 72

'''

pitch_map = {
    0: 52,
    1: 53,
    2: 57,
    3: 59,
    4: 60,
    5: 64,
    6
}

conway = ConwayGenerator(init_state)

states = conway.generate_states(1)

for state in states:
    print(conway.format_state(conway.get_transformed_state(state)))
    print("---------------------------------------------\n")


OOOOOOOOOOXOOOOO
OOOOOOOOOXOXOOOO
OOOOOOOOXOOOXOOO
OOOOOOOXOOOOOXOO
OOOOOOXOOOOOOOXO
OOOOOXOOOOOOOOOX
OOOOXOOOOOOOOOOO
OOOXOOOOOOOOOOOO
OOXOOOOOOOOOOOOO
OXOOOOOOOOOOOOOO
XOOOOOOOOOOOOOOO
---------------------------------------------

